# Codigo final
Este script reúne de manera integrada la aplicabilidad del proyecto. En él se importa el modelo previamente entrenado y se realizan los cálculos necesarios para obtener tanto los valores de NPK como el índice NDVI, lo que permite combinar la información espectral con los datos de nutrientes. A partir de estos resultados se generan visualizaciones que facilitan la interpretación de la distribución de nutrientes en el terreno y el estado vegetativo del cultivo, por ende aproximar el diagnostico del estado de salud de este.

In [ ]:


# Importar la librería joblib
import joblib

#Ruta de archivo
file_path = '/content/drive/MyDrive/random_forest_z_outliers_model.joblib'

# Cargar el archivo joblib
try:
    loaded_object = joblib.load(file_path)
    print("Archivo joblib cargado exitosamente:")

except FileNotFoundError:
    print(f"Error: El archivo no se encontró en la ruta especificada: {file_path}")
except Exception as e:
    print(f"Ocurrió un error al cargar el archivo: {e}")

Archivo joblib cargado exitosamente:


In [ ]:

import os
import re
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px

# ---------- funcs util ----------
def _normalize(s: str) -> str:
    s = str(s).strip().lower()
    s = re.sub(r'[áàäâ]', 'a', s)
    s = re.sub(r'[éèëê]', 'e', s)
    s = re.sub(r'[íìïî]', 'i', s)
    s = re.sub(r'[óòöô]', 'o', s)
    s = re.sub(r'[úùüû]', 'u', s)
    s = s.replace('ñ', 'n')
    s = re.sub(r'[^a-z0-9 _]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def mapear_columnas_por_patron(cols):
    norm2orig = { _normalize(c): c for c in cols }
    norms = list(norm2orig.keys())
    mapping = {}

    patterns = {
        "n": [r'\bn\b', 'nitrogen', 'nitrogeno', 'nitr'],
        "p": [r'\bp\b', 'fosfor', 'fosforo', 'phosph'],
        "k": [r'\bk\b', 'potas', 'potasio', 'potassium'],
        "latitud": ['latitud', r'\blat\b', 'latitude', 'ycoord', 'y_coord'],
        "longitud": ['longitud', r'\blon\b', r'\blng\b', 'longitude', 'xcoord', 'x_coord']
    }

    used = set()
    for key, pats in patterns.items():
        found = None
        for p in pats:
            for nc in norms:
                try:
                    if re.search(p, nc):
                        if nc not in used:
                            found = nc
                            break
                except re.error:
                    continue
            if found:
                break
        if found:
            mapping[key] = norm2orig[found]
            used.add(found)
    return mapping

def coerce_numeric_fix_commas(s: pd.Series):
    s0 = pd.to_numeric(s, errors='coerce')
    n_numeric = s0.notnull().sum()
    s_comm = s.astype(str).str.replace(r'\s+', '', regex=True).str.replace(',', '.', regex=False)
    s1 = pd.to_numeric(s_comm, errors='coerce')
    n_numeric_comm = s1.notnull().sum()
    return s1 if n_numeric_comm >= n_numeric else s0

def leer_archivo_con_diagnostico(ruta):
    if not os.path.exists(ruta):
        raise FileNotFoundError(f"No se encontró el archivo: {ruta}")
    ext = os.path.splitext(ruta)[1].lower()
    if ext == '.csv':
        # intentar con distintos separadores comunes
        for sep in [',',';','\t']:
            try:
                df = pd.read_csv(ruta, sep=sep, engine='python', encoding='utf-8')
                if df.shape[0] >= 1 and df.shape[1] >= 2:
                    # asumimos que detectó algo razonable
                    print(f"Lectura CSV con separador '{sep}' -> shape {df.shape}")
                    return df
            except Exception:
                continue
        # fallback
        df = pd.read_csv(ruta, engine='python', encoding='latin1')
    elif ext in ('.xlsx', '.xls'):
        # probar headers 0..3 por si encabezado está desplazado
        for header in range(0,4):
            try:
                df = pd.read_excel(ruta, header=header)
                if df.shape[0] >= 1 and df.shape[1] >= 2:
                    print(f"Lectura Excel con header={header} -> shape {df.shape}")
                    return df
            except Exception:
                continue
        # fallback
        df = pd.read_excel(ruta)
    else:
        raise ValueError("Formato no soportado. Use .csv o .xlsx/.xls")
    return df

# ---------- MENU: archivo o manual ----------
print("Modo de entrada de datos:")
print("1. Subir archivo (CSV/XLSX) por ruta (file path)")
print("2. Ingresar puntos manualmente")
modo = input("Elige 1 o 2 [1]: ").strip() or "1"

if modo == "1":
    ruta = input("Ruta completa del archivo (ej: /content/datos.xlsx): ").strip()
    df_raw = leer_archivo_con_diagnostico(ruta)
else:
    # ingreso manual
    npt = int(input("Número de puntos a ingresar manualmente: ").strip())
    rows = []
    for i in range(npt):
        print(f"Punto {i+1}:")
        lat = float(input("  Latitud: ").strip())
        lon = float(input("  Longitud: ").strip())
        n = float(input("  N: ").strip())
        p = float(input("  P: ").strip())
        k = float(input("  K: ").strip())
        rows.append({'latitud': lat, 'longitud': lon, 'N': n, 'P': p, 'K': k})
    df_raw = pd.DataFrame(rows)

# ---------- diagnóstico inicial ----------
print("\n=== Diagnóstico inicial del archivo ===")
print("Shape (filas, columnas):", df_raw.shape)
print("Columnas detectadas:", list(df_raw.columns))
print("\nPrimeras filas:")
display(df_raw.head(8))
print("\nConteo de nulos por columna:")
print(df_raw.isnull().sum())

# ---------- mapear columnas N,P,K,lat,long ----------
mapping = mapear_columnas_por_patron(df_raw.columns)
print("\nMapeo detectado (si vacío, revisar encabezados):", mapping)

if set(mapping.keys()) < {"n","p","k","latitud","longitud"}:
    raise SystemExit("No se detectaron todas las columnas necesarias (N,P,K,latitud,longitud). "
                     "Mira el listado de columnas arriba y ajusta nombres o pasa aquí el output para que lo revise.")

# renombrar a nombres estándar
df = df_raw.rename(columns={
    mapping['n']: 'N',
    mapping['p']: 'P',
    mapping['k']: 'K',
    mapping['latitud']: 'Latitud',
    mapping['longitud']: 'Longitud'
})

# forzar numeric (corrige coma decimal si mejora)
for c in ['Latitud','Longitud','N','P','K']:
    df[c] = coerce_numeric_fix_commas(df[c])

print("\nDespués de forzar numéricos:")
print(df[['Latitud','Longitud','N','P','K']].head(8))
print("\nConteo de nulos tras coerción:")
print(df[['Latitud','Longitud','N','P','K']].isnull().sum())

#Verificar
before = len(df_raw)
df_clean = df.dropna(subset=['Latitud','Longitud','N','P','K']).reset_index(drop=True)
after = len(df_clean)
print(f"\nFilas originales: {before}. Filas válidas tras limpieza: {after}. Filas eliminadas: {before-after}")
if after == 0:
    raise SystemExit("No hay filas válidas para procesar después de la limpieza. Revisa el archivo.")
if after < before:
    print("Ejemplo de filas eliminadas (primeras 5):")
    display(df.loc[~df.index.isin(df_clean.index)].head(5))

# ahora df_ref es df_clean que usaremos como puntos muestreo
df_ref = df_clean.copy()
print(f"\nUsaremos {len(df_ref)} puntos de referencia (deben ser tus 19).")

# ---------- Parámetros malla e interpolación IDW ----------
paso = float(input("Ingrese paso para la malla (ej. 0.00001) [por defecto 0.00001]: ") or 0.00001)
power = 2.0

min_lat, max_lat = df_ref['Latitud'].min(), df_ref['Latitud'].max()
min_lon, max_lon = df_ref['Longitud'].min(), df_ref['Longitud'].max()

# crear rangos con margen mínimo para incluir max
cordx = np.arange(min_lon, max_lon + paso, paso)
cordy = np.arange(min_lat, max_lat + paso, paso)
nx, ny = len(cordx), len(cordy)
print(f"Malla generada: {nx} columnas x {ny} filas = {nx*ny} puntos")

# preasignar matrices
matz_N = np.zeros((ny, nx), dtype=float)
matz_P = np.zeros((ny, nx), dtype=float)
matz_K = np.zeros((ny, nx), dtype=float)

ref_lat = df_ref['Latitud'].values
ref_lon = df_ref['Longitud'].values
ref_N = df_ref['N'].values
ref_P = df_ref['P'].values
ref_K = df_ref['K'].values

eps = 1e-12
print("\nRealizando interpolación IDW sobre malla (usando TODOS los puntos muestreo)...")
for i in tqdm(range(nx), desc="columnas"):
    x = cordx[i]
    for j in range(ny):
        y = cordy[j]
        # distancia euclidiana (grados). Si tus coordenadas están en grados y quieres metros, habría que reproyectar.
        d = np.sqrt((ref_lat - y)**2 + (ref_lon - x)**2)
        zero_mask = (d == 0)
        if zero_mask.any():
            idx0 = np.where(zero_mask)[0][0]
            matz_N[j, i] = ref_N[idx0]
            matz_P[j, i] = ref_P[idx0]
            matz_K[j, i] = ref_K[idx0]
            continue
        inv = 1.0 / (d + eps)**power
        w = inv / inv.sum()
        matz_N[j, i] = np.dot(ref_N, w)
        matz_P[j, i] = np.dot(ref_P, w)
        matz_K[j, i] = np.dot(ref_K, w)

# ---------- convertir malla a DataFrame ----------
lon_grid, lat_grid = np.meshgrid(cordx, cordy)
df_grid = pd.DataFrame({
    'Latitud': lat_grid.ravel(),
    'Longitud': lon_grid.ravel(),
    'IDW_N': matz_N.ravel(),
    'IDW_P': matz_P.ravel(),
    'IDW_K': matz_K.ravel()
})
df_grid = df_grid.dropna(subset=['IDW_N','IDW_P','IDW_K']).reset_index(drop=True)
print(f"\nPuntos interpolados válidos generados: {len(df_grid)}")

# ---------- normalizar coords y distancia ----------
coords = df_grid[['Latitud','Longitud']].values
scaler_coords = MinMaxScaler()
coords_norm = scaler_coords.fit_transform(coords)
df_grid['lat_norm'] = coords_norm[:,0]
df_grid['lon_norm'] = coords_norm[:,1]

centroid_lat = df_grid['lat_norm'].mean()
centroid_lon = df_grid['lon_norm'].mean()
df_grid['Distancia'] = np.sqrt((df_grid['lat_norm'] - centroid_lat)**2 + (df_grid['lon_norm'] - centroid_lon)**2)
df_grid['Distancia_metros'] = MinMaxScaler().fit_transform(df_grid[['Distancia']])

# cargar modelo y predecir
if 'file_path' not in globals() and 'file_path' not in locals():
    raise SystemExit("La variable 'file_path' con la ruta del modelo debe estar definida previamente (ej: file_path = '/content/mi_modelo.joblib').")

model_path = file_path
print(f"\nCargando modelo desde: {model_path}")
try:
    model = joblib.load(model_path)
except Exception as e:
    raise SystemExit(f"No se pudo cargar el modelo: {e}")

input_cols = ['IDW_N','IDW_P','IDW_K','Distancia_metros']
missing = [c for c in input_cols if c not in df_grid.columns]
if missing:
    raise SystemExit(f"Faltan columnas requeridas para el modelo: {missing}")

print("\nRealizando predicciones NDVI sobre cada punto de la malla...")
df_grid['NDVI_predicho'] = model.predict(df_grid[input_cols])

# guardar resultados
out_dir = "/content/Entregable"
os.makedirs(out_dir, exist_ok=True)
out_csv = os.path.join(out_dir, "grid_predicciones_ndvi.csv")
df_grid.to_csv(out_csv, index=False)
print(f"\n✅ Resultados guardados en: {out_csv}")
print("\nPrimeras filas del resultado:")
display(df_grid.head())

# Graficos NPK
def plot_contour(z_matrix, x, y, original_df, z_title, out_html=None):
    fig = go.Figure()
    fig.add_trace(go.Contour(z=z_matrix, x=x, y=y, colorscale="Viridis",
                             colorbar=dict(title=z_title), line_smoothing=1))
    if original_df is not None and {"Longitud","Latitud"}.issubset(original_df.columns):
        fig.add_trace(go.Scatter(x=original_df["Longitud"], y=original_df["Latitud"],
                                 mode="markers", marker=dict(color="black", size=6), name="Muestreo"))
    fig.update_layout(width=900, height=700, title=z_title, xaxis_title="Longitud", yaxis_title="Latitud", template="plotly_white")
    if out_html:
        try:
            fig.write_html(out_html)
        except Exception as e:
            print(f"Advertencia al guardar {out_html}: {e}", file=sys.stderr)
    fig.show()
    return fig

plot_contour(matz_N, cordx, cordy, df_ref, "Interpolación IDW - N (mg/kg)", out_html=os.path.join(out_dir,"contour_IDW_N.html"))
plot_contour(matz_P, cordx, cordy, df_ref, "Interpolación IDW - P (mg/kg)", out_html=os.path.join(out_dir,"contour_IDW_P.html"))
plot_contour(matz_K, cordx, cordy, df_ref, "Interpolación IDW - K (mg/kg)", out_html=os.path.join(out_dir,"contour_IDW_K.html"))

# NDVI y genera gráfico aproximado
try:
    ndvi_mat = df_grid['NDVI_predicho'].values.reshape((ny, nx))
    plot_contour(ndvi_mat, cordx, cordy, df_ref, "NDVI predicho (modelo)", out_html=os.path.join(out_dir,"contour_NDVI.html"))
except Exception as e:
    print("No se pudo rearmar matriz NDVI (reshape), se mostrará scatter:", e)
    fig = px.scatter(df_grid, x="Longitud", y="Latitud", color="NDVI_predicho",
                     color_continuous_scale="Viridis", title="NDVI predicho (scatter)", width=900, height=700)
    fig.update_traces(marker=dict(size=3))
    fig.update_layout(template="plotly_white")
    try:
        fig.write_html(os.path.join(out_dir,"scatter_NDVI.html"))
    except Exception as e2:
        print("Advertencia al guardar scatter:", e2)
    fig.show()

print("\nProceso finalizado.")


Modo de entrada de datos:
1. Subir archivo (CSV/XLSX) por ruta (file path)
2. Ingresar puntos manualmente
Elige 1 o 2 [1]: 1
Ruta completa del archivo (ej: /content/datos.xlsx): /content/cultivo mixto.xlsx
Lectura Excel con header=0 -> shape (19, 9)

=== Diagnóstico inicial del archivo ===
Shape (filas, columnas): (19, 9)
Columnas detectadas: ['Latitud ', 'Longitud ', 'pH ', 'N', 'P', 'K', 'Conductuvidad', 'T', 'Humedad ']

Primeras filas:


,Latitud,Longitud,pH,N,P,K,Conductuvidad,T,Humedad
0,1.158627,-77.277333,7.49,15,21,53,200,17.7,19.7
1,1.158649,-77.277330,7.61,10,14,36,100,20.3,8.1
2,1.158669,-77.277329,7.53,15,21,53,200,16.9,32.0
3,1.158692,-77.277334,7.31,13,18,47,161,16.0,25.0
4,1.158687,-77.277264,7.39,10,14,36,100,17.4,14.1
5,1.158665,-77.277264,7.32,15,21,53,200,17.8,13.8
6,1.158642,-77.277263,7.73,22,30,75,326,15.6,31.3
7,1.158625,-77.277264,7.52,15,21,53,200,17.4,13.4



Conteo de nulos por columna:
Latitud          0
Longitud         0
pH               0
N                0
P                0
K                0
Conductuvidad    0
T                0
Humedad          0
dtype: int64

Mapeo detectado (si vacío, revisar encabezados): {'n': 'N', 'p': 'P', 'k': 'K', 'latitud': 'Latitud ', 'longitud': 'Longitud '}

Después de forzar numéricos:
    Latitud   Longitud   N   P   K
0  1.158627 -77.277333  15  21  53
1  1.158649 -77.277330  10  14  36
2  1.158669 -77.277329  15  21  53
3  1.158692 -77.277334  13  18  47
4  1.158687 -77.277264  10  14  36
5  1.158665 -77.277264  15  21  53
6  1.158642 -77.277263  22  30  75
7  1.158625 -77.277264  15  21  53

Conteo de nulos tras coerción:
Latitud     0
Longitud    0
N           0
P           0
K           0
dtype: int64

Filas originales: 19. Filas válidas tras limpieza: 19. Filas eliminadas: 0

Usaremos 19 puntos de referencia (deben ser tus 19).
Ingrese paso para la malla (ej. 0.00001) [por defecto 0.00001]: 0.0

columnas: 100%|██████████| 27/27 [00:00<00:00, 2366.05it/s]


Puntos interpolados válidos generados: 297

Cargando modelo desde: /content/drive/MyDrive/random_forest_z_outliers_model.joblib



Realizando predicciones NDVI sobre cada punto de la malla...

✅ Resultados guardados en: /content/Entregable/grid_predicciones_ndvi.csv

Primeras filas del resultado:


,Latitud,Longitud,IDW_N,IDW_P,IDW_K,lat_norm,lon_norm,Distancia,Distancia_metros,NDVI_predicho
0,1.158601,-77.277334,14.486601,20.188192,51.120806,0.0,0.000000,0.707107,1.000000,0.257758
1,1.158601,-77.277324,14.562667,20.279571,51.348374,0.0,0.038462,0.680454,0.962307,0.257758
2,1.158601,-77.277314,14.700606,20.445097,51.763290,0.0,0.076923,0.654976,0.926277,0.259234
3,1.158601,-77.277304,14.933607,20.740002,52.490910,0.0,0.115385,0.630816,0.892109,0.261017
4,1.158601,-77.277294,15.213873,21.106565,53.385822,0.0,0.153846,0.608130,0.860026,0.262019



Proceso finalizado.
